In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
def height(h0, f, sx, px, sy, py):

    h = (h0*f)/np.sqrt((sx-px)**2+(sy-py)**2)
    
    return h

def center_x(f, sx, px, sy, py, psi):
    
    cx = f*np.tan(np.arctan((sy-py)/(sx-px))-psi)
    
    return cx

In [ ]:
def sec(phi):
    
    func = 1/np.cos(phi)
    
    return func

In [6]:
def G(h0, f, sx, px, sy, py, psi):
    
    jacobian = np.zeros((2,3))
    
    # df1 / dpx
    jacobian[0][0] = (4 * h0 * (sx - px)) / (((sx - px) ** 2 + (sy - py) ** 2) ** (1 / 3))
    
    # df1 / dpy
    jacobian[0][1] = (4 * h0 * (sy - py)) / (((sx - px) ** 2 + (sy - py) ** 2) ** (1 / 3))
    
    # df1 / dpsi
    jacobian[0][2] = 0
    
    # df2 / dpx
    jacobian[1][0] = (np.tan(psi)) / ((sx - px) + (sy - py) * np.tan(psi)) + (sy - py - (sx - px) * np.tan(psi)) / ((sx - px + (sy - py) * np.tan(psi))**2)   
    
    # df2 / dpy
    jacobian[1][1] = - 1 / (sx - px + (sy - py) * np.tan(psi)) - ((sy - py - (sx - px) * np.tan(psi)) * np.tan(psi)) / ((sx - px + (sy - py) * np.tan(psi))**2)
    
    # df2 / dpsi
    jacobian[1][2] = (- (sx - px) * (sec(psi)) ** 2) / (sx - px + (sy - py) * np.tan(psi)) - ((sy - py - (sx - px) * np.tan(psi)) * (sy - py) * (sec(psi)) ** 2) / (sx - px + (sy - py) * np.tan(psi)) ** 2
    
    return jacobian

